In [12]:
# Import des datasets et suppression des colonnes Unnamed
import pandas as pd 

dfCarac = pd.read_csv("20221010_table_caracteristiques.csv")
dfLieux = pd.read_csv("20221010_table_lieux.csv")
dfVehicules = pd.read_csv("20221010_table_vehicules.csv")


C:\Users\benpa\AppData\Local\Temp\ipykernel_9344\3686088640.py:4: DtypeWarning: Columns (6,12,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  dfCarac = pd.read_csv("20221010_table_caracteristiques.csv")
C:\Users\benpa\AppData\Local\Temp\ipykernel_9344\3686088640.py:5: DtypeWarning: Columns (4,9,10,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  dfLieux = pd.read_csv("20221010_table_lieux.csv")
C:\Users\benpa\AppData\Local\Temp\ipykernel_9344\3686088640.py:6: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  dfVehicules = pd.read_csv("20221010_table_vehicules.csv")


In [13]:
print(len(dfCarac.groupby(["Num_Acc"]).count()))
print(len(dfCarac))

1065053
1065053


In [14]:
# Création des nouvelles variables pour l'étude de la gravité des accidents de la route 
# On rapatrie la commune liée à l'accident et sa population 
dfCommunes = pd.read_csv("pop_commune.csv", sep=";")

# Définition fonction de mise au format département
def mise_au_format_dep(df) : 
    df["dep"] = df["dep"].astype(str)
    df["dep"] = df["dep"].apply(lambda x: x[:2] if (len(x) == 3 and x[2] == "0") else x)
    df["dep"] = df["dep"].apply(lambda x: x[1:] if (x[0] == "0") else x)
    df["dep"] = df["dep"].apply(lambda x: "0" + x if (len(x) == 1) else x)
    df["dep"] = df["dep"].replace("201", "2A")
    df["dep"] = df["dep"].replace("202", "2B")

# Application aux tables dfCarac et dfCommunes
mise_au_format_dep(dfCarac)
mise_au_format_dep(dfCommunes)

# Travail sur la colonne "com" de la table dfCarac pour la mettre au format
dfCarac["com"] = pd.to_numeric(dfCarac["com"], errors= "coerce")

dfCarac["com"] = dfCarac["com"].apply(lambda x: x % 100 if x > 1000 else x)
dfCommunes["com"] = pd.to_numeric(dfCommunes["com"], errors= "coerce")
dfCommunes = dfCommunes.drop(["code_region", "code_arrondissement", "code_canton", "population_mun", 'population_part'], axis = 1)
dfCommunes = dfCommunes.drop_duplicates(subset= ["dep", "com"], keep = 'first')
dfCarac = dfCarac.merge(dfCommunes, on = ["dep", "com"], how = "left")
dfCarac.head()


,Unnamed: 0.1,Unnamed: 0,Num_Acc,an,mois,jour,hrmn,lum,agg,int,...,com,adr,gps,lat,long,dep,grav,nom_region,nom_commune,population_tot
0,0,0,201600000001,16,2,1,1445,1,2,1,...,5.0,"46, rue Sonneville",M,0.0,0.0,59,3,Hauts-de-France,Allennes-les-Marais,3 515
1,1,1,201600000002,16,3,16,1800,1,2,6,...,5.0,1a rue du cimetière,M,0.0,0.0,59,3,Hauts-de-France,Allennes-les-Marais,3 515
2,2,2,201600000003,16,7,13,1900,1,1,1,...,11.0,NaN,M,0.0,0.0,59,3,Hauts-de-France,Annœullin,10 583
3,3,3,201600000004,16,8,15,1930,2,2,1,...,477.0,52 rue victor hugo,M,0.0,0.0,59,3,Hauts-de-France,Provin,4 443
4,4,4,201600000005,16,12,23,1100,1,2,3,...,11.0,rue Joliot curie,M,0.0,0.0,59,3,Hauts-de-France,Annœullin,10 583


In [15]:
print("Le pourcentage de NA est de ", round(dfCarac["nom_region"].isna().sum() / dfCarac["nom_region"].size , 3) * 100)
print(dfCarac.nom_commune.value_counts()[30:50])
print(len(dfCarac))

Le pourcentage de NA est de  3.3000000000000003
Aulnay-sous-Bois       2839
Nancy                  2836
Saint-Étienne          2772
Saint-Pierre           2743
Champigny-sur-Marne    2723
Aix-en-Provence        2701
Besançon               2619
Argenteuil             2606
Le Mans                2554
Aubervilliers          2476
Montreuil              2341
Colombes               2326
Roubaix                2272
Le Lamentin            2200
Dijon                  2196
La Courneuve           2184
Le Havre               2124
Courbevoie             2106
Les Pennes-Mirabeau    2101
Ajaccio                2097
Name: nom_commune, dtype: int64
1065053


In [16]:
dfCarac['population_tot'] = dfCarac['population_tot'].str.replace(" ", "")
dfCarac['population_tot'] = pd.to_numeric(dfCarac['population_tot'])

def regroupement_population(x) :
    if x < 2000 :
        return 'Village'
    elif x < 5000 :
        return 'Bourg'
    elif x < 20000 :
        return 'Petite Ville'
    elif x < 50000 :
        return 'Ville Moyenne'
    elif x < 200000 :
        return 'Grande Ville'
    else :
        return "Métropole"
    
dfCarac['populationGrp'] = dfCarac['population_tot'].apply(lambda x: regroupement_population(x))
print(dfCarac.head())


   Unnamed: 0.1  Unnamed: 0       Num_Acc  an  mois  jour  hrmn  lum  agg  \
0             0           0  201600000001  16     2     1  1445    1    2   
1             1           1  201600000002  16     3    16  1800    1    2   
2             2           2  201600000003  16     7    13  1900    1    1   
3             3           3  201600000004  16     8    15  1930    2    2   
4             4           4  201600000005  16    12    23  1100    1    2   

   int  ...                  adr  gps  lat long dep grav       nom_region  \
0    1  ...   46, rue Sonneville    M  0.0  0.0  59    3  Hauts-de-France   
1    6  ...  1a rue du cimetière    M  0.0  0.0  59    3  Hauts-de-France   
2    1  ...                  NaN    M  0.0  0.0  59    3  Hauts-de-France   
3    1  ...   52 rue victor hugo    M  0.0  0.0  59    3  Hauts-de-France   
4    3  ...     rue Joliot curie    M  0.0  0.0  59    3  Hauts-de-France   

           nom_commune  population_tot populationGrp  
0  Allennes-les-Mar

In [17]:
# Regroupement des types d'intersection

def regroupement_intersection(x) :
    if x == 1 :
        return "Hors intersection"
    elif x in [2,3,4] :
        return "Croisement de deux routes"
    elif x in [5,6,7] :
        return "Croisement circulaire" 
    elif x == 8 :
        return "Passage à niveau"
    else : 
        return "Autres"

dfCarac["intGrp"] = dfCarac.int.apply(regroupement_intersection)

In [18]:
# On garde la liste des variables suivantes 
# On filtre les accidents qui ne se sont pas produits à Bornel, Betz ou Auneuil qui sont dans le TOP30

dfCarac_filtre = dfCarac[(dfCarac.nom_commune != "Bornel") & (dfCarac.nom_commune != "Betz") & (dfCarac.nom_commune != "Auneuil")]
variable_conservees = ["Num_Acc", "populationGrp", "intGrp"]
dfCarac_filtre = dfCarac_filtre[variable_conservees]
print(dfCarac_filtre.head())
print(1-len(dfCarac_filtre)/len(dfCarac))

        Num_Acc populationGrp                     intGrp
0  201600000001         Bourg          Hors intersection
1  201600000002         Bourg      Croisement circulaire
2  201600000003  Petite Ville          Hors intersection
3  201600000004         Bourg          Hors intersection
4  201600000005  Petite Ville  Croisement de deux routes
0.0187361567921972


In [19]:
# Variables à conserver sur la table Lieux 
variable_conservees = ["Num_Acc", "catr", "circ"]
dfLieux_filtre = dfLieux[variable_conservees]
dfLieux_filtre.head()

,Num_Acc,catr,circ
0,201600000001,3.0,2.0
1,201600000002,3.0,1.0
2,201600000003,3.0,2.0
3,201600000004,4.0,2.0
4,201600000005,4.0,0.0


In [20]:
# Travaux sur la table véhicules
print(dfVehicules.dtypes)
def regroupement_cat_veh(x) :
    cat1 = [1, 2, 4, 5, 6, 30, 31, 32, 33,34, 80, 35, 36, 41, 42, 43] # 2, 3 roues et quads
    cat2 = [3, 7, 8, 9, 10, 11, 12] # VL et VUL
    cat3 = [13, 14, 15, 16, 17, 20, 21, 37, 38, 18] # PL 
    cat4 = [39] # Trains
    cat5 = [40, 19, 99, 0] # Tramways
    cat9 = [50,60]
    if x in cat1 :
        return "2,3 roues & quads"
    elif x in cat2 :
        return "VL & VUL"
    elif x in cat3 :
        return "PL"
    elif x in cat4 :
        return "Train"
    elif x in cat5:
        return "Autres"
    elif x in cat9 :
        return "EPD"
    
dfVehicules["catvGrp"] = dfVehicules.catv.apply(regroupement_cat_veh)

def regroupement_obstacles(x) :
    if x == 0 :
        return "Pas d'Obstacle"
    elif x == - 1 :
        return "Z"
    else : 
        return "Obstacle" 

dfVehicules["obsGrp"] = dfVehicules.obs.apply(regroupement_obstacles)



Unnamed: 0.1      int64
Unnamed: 0        int64
Num_Acc           int64
senc            float64
catv              int64
occutc          float64
obs             float64
obsm            float64
choc            float64
manv            float64
num_veh          object
id_vehicule      object
motor           float64
grav              int64
dtype: object


In [21]:
# Construction de la variable au moins un obstacle dans l'accident 
dfVehicules_obs = dfVehicules.sort_values(by = 'obs', ascending = True)
dfVehicules_obs = dfVehicules_obs.drop_duplicates(subset= ["Num_Acc"], keep= 'first')
dfVehicules_obs_filtre = dfVehicules_obs[['Num_Acc', "obsGrp"]]
print(len(dfVehicules_obs)) 
print(dfVehicules_obs_filtre["obsGrp"].value_counts())

1065053
Pas d'Obstacle    892899
Obstacle          172093
Z                     61
Name: obsGrp, dtype: int64


In [22]:
# Construction de la variable nombre de véhicules
dfNombreVehicule = dfVehicules[['Num_Acc',"num_veh"]].groupby(['Num_Acc']).count().reset_index()
dfNombreVehicule = dfNombreVehicule.sort_values(by='num_veh', ascending = False) 

def regroupement_nb_vehicules(x) :
    if x == 1 :
        return "1 véhicule"
    elif x == 2 :
        return "2 véhicules"
    elif x == 3 :
        return "3 véhicules"
    elif x > 3 and x < 10 :
        return "entre 4 et 10 véhicules"
    else : 
        return "+ de 10 véhicules" 

dfNombreVehicule["nbVeh"] = dfNombreVehicule.num_veh.apply(regroupement_nb_vehicules)

print(dfNombreVehicule.head())

             Num_Acc  num_veh              nbVeh
161942  200600074917       56  + de 10 véhicules
526968  201100052545       54  + de 10 véhicules
103859  200600016834       44  + de 10 véhicules
783025  201600002473       35  + de 10 véhicules
353046  200900022411       28  + de 10 véhicules


In [28]:
# Construction des variables 'catv_train_exist', 'catv_PL_exist', 'catv_2_roues_exist', 'catv_epdm_exist'
import numpy as np
dfVehicules['catv_train_exist'] = np.where(dfVehicules.catvGrp =="Train", 1,0)
dfVehicules['catv_PL_exist'] = np.where(dfVehicules.catvGrp =="PL", 1,0)
dfVehicules['catv_2_roues_exist'] = np.where(dfVehicules.catvGrp =="2,3 roues & quads", 1,0)
dfVehicules['catv_EPD_exist'] = np.where(dfVehicules.catvGrp =="EPD", 1,0)

dfVeh_type_veh = dfVehicules[['Num_Acc', 'catv_train_exist', "catv_PL_exist", "catv_2_roues_exist", "catv_EPD_exist"]].groupby("Num_Acc").sum().reset_index()

dfVeh_type_veh["catv_train_exist"] = dfVeh_type_veh["catv_train_exist"].apply(lambda x : 1 if x >=1 else 0)
dfVeh_type_veh["catv_PL_exist"] = dfVeh_type_veh["catv_PL_exist"].apply(lambda x : 1 if x >=1 else 0)
dfVeh_type_veh["catv_2_roues_exist"] = dfVeh_type_veh["catv_2_roues_exist"].apply(lambda x : 1 if x >=1 else 0)
dfVeh_type_veh["catv_EPD_exist"] = dfVeh_type_veh["catv_EPD_exist"].apply(lambda x : 1 if x >=1 else 0)

dfVeh_type_veh.head()


,Num_Acc,catv_train_exist,catv_PL_exist,catv_2_roues_exist,catv_EPD_exist
0,200500000001,0,0,0,0
1,200500000002,0,0,1,0
2,200500000003,0,0,1,0
3,200500000004,0,0,0,0
4,200500000005,0,0,0,0
